## Simple genai app using langchain

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [3]:
# data ingestion - scrape data for web
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.v7labs.com/blog/vision-transformer-guide")
loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
docs =loader.load()
docs

[Document(metadata={'source': 'https://www.v7labs.com/blog/vision-transformer-guide', 'title': 'Vision Transformer: What It Is & How It Works [2024 Guide]', 'description': 'A vision transformer (ViT) is a transformer-like model that handles vision processing tasks. Learn how it works and see some examples.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\nVision Transformer: What It Is & How It Works [2024 Guide]\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nV7 Go introduces Knowledge HubsWatch the announcementProductsResourcesCompanySign upLog inBlogWebinarsAI agentsDarwin academyResourcesBook a demoBook a demoBook a demoComputer visionVision Transformer: What It Is & How It Works [2024 Guide]\n            14\n             min read—Dec 15, 2022A vision transformer (ViT) is a transformer-like model that handles vision processing tasks. Learn how it works and see some examples.Deval ShahGuest Author6:01NEW - V7 Go Product UpdateIntroducing Knowledge HubsPlay vi

In [7]:
## load data -> docs -> divide into chunks -> vector embeddings -> vector store db
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
document = splitter.split_documents(docs)

In [8]:
document

[Document(metadata={'source': 'https://www.v7labs.com/blog/vision-transformer-guide', 'title': 'Vision Transformer: What It Is & How It Works [2024 Guide]', 'description': 'A vision transformer (ViT) is a transformer-like model that handles vision processing tasks. Learn how it works and see some examples.', 'language': 'en'}, page_content='Vision Transformer: What It Is & How It Works [2024 Guide]'),
 Document(metadata={'source': 'https://www.v7labs.com/blog/vision-transformer-guide', 'title': 'Vision Transformer: What It Is & How It Works [2024 Guide]', 'description': 'A vision transformer (ViT) is a transformer-like model that handles vision processing tasks. Learn how it works and see some examples.', 'language': 'en'}, page_content='V7 Go introduces Knowledge HubsWatch the announcementProductsResourcesCompanySign upLog inBlogWebinarsAI agentsDarwin academyResourcesBook a demoBook a demoBook a demoComputer visionVision Transformer: What It Is & How It Works [2024 Guide]\n          

In [16]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L12-v2"
)

d:\langchain\lenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP down

In [17]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(document,embeddings)

In [18]:
vectorstoredb

In [19]:
query = "Transformers are non-sequential, in contrast to RNNs that take in sequential data. "
result = vectorstoredb.similarity_search(query)
result[0].page_content

"projections.Yet another problem with Transformers is that the sequence order is not enforced naturally since data is passed in at a shot instead of timestep-wise, as is done in RNNs and LSTMs. To combat this, the original Transformer paper suggests using Positional Encodings/Embeddings that establish a certain order in the inputs.The positional embedding matrix\xa0Epos is randomly generated and added to the concatenated matrix containing the learnable class embedding and patch projections.D is the fixed latent vector size used throughout the Transformer. It’s what we squash the input vectors to before passing them into the Encoder.Altogether, these patch projections and positional embeddings form a larger matrix that’ll soon be put through the Transformer Encoder.The outputs of the Transformer Encoder are then sent into a Multilayer Perceptron for image classification. The input features capture the image's essence very well, making the MLP head’s classification task far simpler.Pro t

In [20]:
from langchain_groq import ChatGroq

# Replace with a supported model
llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.7, max_tokens=1024)

In [21]:
# retrieval chain , document chain 
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
answer the fowllowing based on provided context:
<context>
{context}
<context>

"""
    
)

document_chain = create_stuff_documents_chain(llm , prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nanswer the fowllowing based on provided context:\n<context>\n{context}\n<context>\n\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020185876BA0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020185877DA0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'), max_tokens=1024)
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [27]:
from langchain_core.documents import Document

response = document_chain.invoke({
    "input": "Transformers are non-sequential, in contrast to RNNs that take in sequential",
    "context": [Document(page_content="Transformers are non sequential, in contrast to RNNs that take in sequential data.For example, if my input is a sentence, RNNs will take one word at a time as input. This isn't the case for transformers—they can take all the words of the sentence as input.")]
})

print(response)

I'll answer your questions based on the provided context about Transformers and RNNs. What's your question?


In [30]:
#retriever
ret = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
ret_chain = create_retrieval_chain(ret,document_chain)

In [35]:
response= ret_chain.invoke({"input":"Transformers are non-sequential, in contrast to RNNs that take in sequential"})
response['answer']

"I'm ready to answer your questions based on the provided context about Transformers, Vision Transformers, and their applications. What's your question?"

In [37]:
response


{'input': 'Transformers are non-sequential, in contrast to RNNs that take in sequential',
 'context': [Document(id='4dd609d0-f206-433a-8809-78e7320c95cf', metadata={'source': 'https://www.v7labs.com/blog/vision-transformer-guide', 'title': 'Vision Transformer: What It Is & How It Works [2024 Guide]', 'description': 'A vision transformer (ViT) is a transformer-like model that handles vision processing tasks. Learn how it works and see some examples.', 'language': 'en'}, page_content="projections.Yet another problem with Transformers is that the sequence order is not enforced naturally since data is passed in at a shot instead of timestep-wise, as is done in RNNs and LSTMs. To combat this, the original Transformer paper suggests using Positional Encodings/Embeddings that establish a certain order in the inputs.The positional embedding matrix\xa0Epos is randomly generated and added to the concatenated matrix containing the learnable class embedding and patch projections.D is the fixed lat

In [38]:
response['context']

[Document(id='4dd609d0-f206-433a-8809-78e7320c95cf', metadata={'source': 'https://www.v7labs.com/blog/vision-transformer-guide', 'title': 'Vision Transformer: What It Is & How It Works [2024 Guide]', 'description': 'A vision transformer (ViT) is a transformer-like model that handles vision processing tasks. Learn how it works and see some examples.', 'language': 'en'}, page_content="projections.Yet another problem with Transformers is that the sequence order is not enforced naturally since data is passed in at a shot instead of timestep-wise, as is done in RNNs and LSTMs. To combat this, the original Transformer paper suggests using Positional Encodings/Embeddings that establish a certain order in the inputs.The positional embedding matrix\xa0Epos is randomly generated and added to the concatenated matrix containing the learnable class embedding and patch projections.D is the fixed latent vector size used throughout the Transformer. It’s what we squash the input vectors to before passi